# Handling Imbalanced Datasets: Beginner’s Guide

Imbalanced datasets are a common challenge in machine learning: one class (or category) in your data is much more frequent than the others. For example, imagine classifying emails as “spam” or “not spam,” but only 1 in 100 is actually spam. If you only train on these numbers, a “lazy” model could just predict “not spam” every time and get 99% accuracy—while missing all the real spam! Here’s how to understand and handle this problem.

## 1. **Recognize Imbalance—and Why Accuracy Can Be Misleading**

- If your model always predicts the majority class, accuracy can look high even though it’s doing a poor job on the minority (important) class.
- **Key takeaway:** Use better metrics (see below).

## 2. **Use Better Evaluation Metrics**

- Instead of plain accuracy, use:

    - **Precision:** Of the items predicted as “positive,” how many were actually positive?
    - **Recall:** Of the actual positives, how many did we catch?
    - **F1-score:** Combines precision and recall into a single number—great for imbalanced data.
- For multiclass problems: look at the **per-class** F1 and “macro-average” (an unweighted mean across classes).

In [4]:
# importing the dependencies
import numpy as np
import pandas as pd

In [9]:
# loading the dataset to pandas DataFrame
credit_card_data = pd.read_csv('credit_data.csv')

In [11]:
# first 5 rows of the dataframe
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [13]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [15]:
# distribution of the two classes
credit_card_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

This is Highly Imbalanced Dataset

0 --> Legit Transactions

1 --> Fraudulent Transactions

In [19]:
# separating the legit and fraudulent transactions
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [21]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


Under-sampling

Build a sample dataset containing similar distribution of Legit & Fraudulent Transactionds

Number of Fraudulent Transactions --> 492

In [26]:
legit_sample = legit.sample(n=492)

In [28]:
print(legit_sample.shape)

(492, 31)


Concatenate the Two DataFrames

In [31]:
new_dataset = pd.concat([legit_sample, fraud], axis = 0)

In [33]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
48980,43861.0,-0.205412,0.096434,1.790543,-1.298015,-0.788676,-0.328766,-0.009984,-0.035475,-1.240959,...,-0.403310,-0.754387,0.047794,-0.109126,-0.456589,0.843609,0.015720,0.059709,35.84,0
163726,116170.0,-0.548753,0.249133,-0.272446,-2.288358,0.022297,-0.879827,0.153325,0.383740,-1.218324,...,0.131127,0.223531,0.014513,-0.431216,-0.603924,-0.533005,0.285345,0.163913,17.00,0
210897,138174.0,-2.508733,2.635981,-2.392639,0.777401,-0.260938,-0.150837,-0.636172,2.061555,-1.204090,...,0.343280,0.471584,-0.068671,0.209933,0.170994,-0.419271,-0.478963,-0.073699,4.70,0
50361,44461.0,1.364291,-0.557719,0.217033,-0.962711,-0.610541,-0.186899,-0.584871,-0.036977,-1.072759,...,-0.075503,-0.403229,-0.020645,-0.527817,0.329961,-0.449597,0.004348,0.010702,40.00,0
114544,73526.0,-0.376049,0.943154,0.845957,-0.609064,1.063875,0.899330,0.525230,0.298539,-0.623868,...,-0.256077,-0.640807,-0.225597,-1.714723,-0.047857,0.188776,0.279387,0.091038,2.69,0


In [35]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [37]:
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64